In [ ]:
import csv
import random
import itertools
import os
import nltk 
from nltk.tokenize import word_tokenize,sent_tokenize
import keras
from keras.optimizers import *
from keras.layers import *
from keras.models import Model, load_model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers 
import numpy as np
from numpy.linalg import cholesky
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [ ]:
with open('DocMeta2.tsv') as f:
    newsdata=f.readlines()


news={}
category={}
subcategory={}
for i in newsdata:
    line=i.strip().split('\t')
    if len(line)<5:
        continue
    news[line[1]]=[line[2],line[3],word_tokenize(line[6].lower())]
    if line[2] not in category:
        category[line[2]]=len(category)
    if line[3] not in subcategory:
        subcategory[line[3]]=len(subcategory)

In [ ]:
with open('UserSQUET.tsv')as f:
    dataq=f.readlines()

with open('ClickData2.tsv')as f:
    user=f.readlines()
userid_dict={}
for i in user:
    line=i.strip().split('\t')
    userid=line[0]
    if userid not in userid_dict:
        userid_dict[userid]=len(userid_dict)

userdata={}
for i in dataq:
    line=i.replace('\n','').split('\t')
    query=[word_tokenize(x.split('#TAB#')[0].lower()) for x in line[2].split('#N#')]
    web=[word_tokenize(x.split('#TAB#')[0].lower()) for x in line[3].split('#N#')]
    userdata[line[0]]=[query,web]

In [ ]:
raw_word_dict={'PADDING':[0,999999]}

for i in news:
    for word in news[i][2]:
        if word in raw_word_dict:
            raw_word_dict[word][1]+=1
        else:
            raw_word_dict[word]=[len(raw_word_dict),1]
for i in userdata:
    for query in userdata[i][0]:
        for word in query:
            if word in raw_word_dict:
                raw_word_dict[word][1]+=1
            else:
                raw_word_dict[word]=[len(raw_word_dict),1]
for i in userdata:
    for title in userdata[i][1]:
        for word in title:
            if word in raw_word_dict:
                raw_word_dict[word][1]+=1
            else:
                raw_word_dict[word]=[len(raw_word_dict),1]

In [ ]:
word_dict={}
for i in raw_word_dict:
    if raw_word_dict[i][1]>=3:
        word_dict[i]=[len(word_dict),raw_word_dict[i][1]]
print(len(word_dict),len(raw_word_dict))



embdict={}
plo=0
import pickle
with open('glove.840B.300d.txt','rb')as f:
    linenb=0
    while True:
        j=f.readline()
        if len(j)==0:
            break
        k = j.split()
        word=k[0].decode()
        linenb+=1
        if len(word) != 0:
            line=[float(x) for x in k[1:]]
            if word in word_dict:
                embdict[word]=line
                if plo%100==0:
                    print(plo,linenb,word)
                plo+=1



word_dict1=word_dict
print(len(embdict),len(word_dict1))
print(len(word_dict1))
embmat=[0]*len(word_dict1)
xp=np.zeros(300,dtype='float32')

cand=[]
for i in embdict.keys():
    embmat[word_dict1[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(embmat[word_dict1[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(embmat)):
    if type(embmat[i])==int:
        embmat[i]=np.reshape(norm, 300)
embmat[0]=np.zeros(300,dtype='float32')
embmat=np.array(embmat,dtype='float32')

In [ ]:
news_words = [[0]*30]
news_index = {'0':0}
for i in news:
    line = []
    news_index[i] = len(news_index)
    for j in news[i][2]:
        if j in word_dict:
            line.append(word_dict[j][0])
    line = line[:30]
    news_words.append(line+[0]*(30-len(line)))

news_words=np.array(news_words,dtype='int32') 



user_query = []
user_title = []
MAXQ=300
MAXT=300
MAXQL=10
MAXTL=25

for i in userid_dict:
    
    if i in userdata:
        tq=[]
        tt=[]
        for j in userdata[i][0]:
            
            tqq=[word_dict[m][0] for m in j if m in word_dict][:MAXQL]
            tq.append(tqq+[0]*(MAXQL-len(tqq)))
        tq=tq[:MAXQ]
        user_query.append(tq+[[0]*MAXQL]*(MAXQ-len(tq)))
        for j in userdata[i][1]:
            
            ttt=[word_dict[m][0] for m in j if m in word_dict][:MAXTL]
            tt.append(ttt+[0]*(MAXTL-len(ttt)))
        tt=tt[:MAXT]
        user_title.append(tt+[[0]*MAXTL]*(MAXT-len(tt)))
    else:
        user_query.append([[0]*MAXQL]*MAXQ)
        user_title.append([[0]*MAXTL]*MAXT)

user_query=np.array(user_query,dtype='int32') 
user_title=np.array(user_title,dtype='int32') 

In [ ]:
def newsample(neglist,ratio):
    if ratio >len(neglist):
        return random.sample(neglist*(ratio//len(neglist)+1),ratio)
    else:
        return random.sample(neglist,ratio)

npratio=4
train_userid=[]
train_candidate=[]    
train_label=[]

test_userid=[]
test_candidate=[]    
test_label=[]
all_test_index=[]

train_history=[]
test_history=[]

for raw in user:
    line=raw.strip().split('\t')
    userid=line[0]
    if len(line)==4:
        
        impr=[x.split('#TAB#') for x in line[2].split('#N#')]
    if len(line)==3:
        impr=[x.split('#TAB#') for x in line[2].split('#N#')]
        

    
    trainpos=[x[0].split() for x in impr]
    trainneg=[x[1].split() for x in impr]
    
 
    poslist=list(itertools.chain(*(trainpos)))
    neglist=list(itertools.chain(*(trainneg)))
    
    
    
    if len(line)==4:
        impr=[x.split('#TAB#') for x in line[3].split('#N#')]
        rawtestpos=[x[0].split() for x in impr] 
        rawtestneg=[x[1].split() for x in impr] 
        
        testpos=[]
        testneg=[]
        for k in range(len(rawtestpos)):
 
            testpos.append(rawtestpos[k])
            testneg.append(rawtestneg[k])
        
        for i in range(len(testpos)):
            currentindex=[]
            currentindex.append(len(test_candidate))
            history=random.sample(poslist,min(50,len(poslist)))[:50]
            history+=[0]*(50-len(history))
    
            for j in testpos[i]:
                test_candidate.append(news_index[j])
                test_label.append(1)
                test_userid.append(userid_dict[userid])
                test_history.append(history)
                
            for j in testneg[i]:
                test_candidate.append(news_index[j])
                test_label.append(0)
                test_userid.append(userid_dict[userid])
                test_history.append(history)
            currentindex.append(len(test_candidate))
            all_test_index.append(currentindex)
            

 
    for posindex in range(len(trainpos)):
        for posdoc in trainpos[posindex]:
            
            candidate_list=newsample(trainneg[posindex],npratio)
            candidate_list.append(posdoc)
            tplb=[0]*npratio+[1]
            tid=list(range(npratio+1))
            random.shuffle(tid)
            yp=[]
            yplb=[]
            for j in tid:
                yp.append(news_index[candidate_list[j]])
                yplb.append(tplb[j])
            
            poslist_reduced=list(set(poslist)-set([posdoc]))
            history=random.sample(poslist_reduced,min(50,len(poslist_reduced)))[:50]
            history+=[0]*(50-len(history))
            train_candidate.append(yp)
            train_label.append(yplb)
            train_userid.append(userid_dict[userid])
            train_history.append(history)



train_candidate=np.array(train_candidate,dtype='int32')
train_label=np.array(train_label,dtype='int32')
train_userid=np.array(train_userid,dtype='int32')
test_candidate=np.array(test_candidate,dtype='int32')
test_label=np.array(test_label,dtype='int32')
test_userid=np.array(test_userid,dtype='int32')
train_history=np.array(train_history,dtype='int32')
test_history=np.array(test_history,dtype='int32')

In [ ]:
def generate_batch_data_random(train_candidate,train_label,train_userid,batch_size):
    idx = np.arange(len(train_label))
    np.random.shuffle(idx)
    y=train_label
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            item = news_words[train_candidate[i]]
            user=news_words[train_history[i]]
            label=train_label[i]
            
            yield ([item,user,user_query[train_userid[i]],user_title[train_userid[i]]], [label])


def generate_batch_data(train_candidate,train_label,train_userid,batch_size):
    idx = np.arange(len(train_label))

    y=train_label
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            item = news_words[train_candidate[i]]
            user=news_words[test_history[i]]
            label=train_label[i]
            
            yield ([item,user,user_query[train_userid[i]],user_title[train_userid[i]]], [label])

In [ ]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)




class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], 1)))
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)

    def call(self, x, mask=None):

        eij = K.dot(x, self.W)
        ai = K.exp(eij)
        weights = ai / K.expand_dims(K.sum(ai, axis=1), 1)
        print('weights', weights.shape)
        weighted_input = x * weights
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [1]:
MAX_SENT_LENGTH=30
MAX_SENTS=50
MAXQ=300
MAXT=300
MAXQL=10
MAXTL=25
results=[]
for npratio in range(4,5):
    keras.backend.clear_session()
    sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    embedding_layer = Embedding(len(word_dict), 300, weights=[embmat],trainable=True) 
    embedded_sequences = embedding_layer(sentence_input)
    d_emb=Dropout(0.2)(embedded_sequences)
    
    news_cnn = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(d_emb)
    d_news_cnn=Dropout(0.2)(news_cnn)
    newsrep=AttLayer()(Dense(200,activation='tanh')(d_news_cnn))
    news_encoder = Model([sentence_input], newsrep)
    news_input = keras.Input((MAX_SENTS,MAX_SENT_LENGTH,), dtype='int32')
    newsreps = TimeDistributed(news_encoder)(news_input)
    userrep_news=AttLayer()(Dense(200,activation='tanh')(newsreps))
    
    query_input = Input(shape=(MAXQL,), dtype='int32')
    query_embedded_sequences = embedding_layer(query_input)
    d_query_embedded_sequences=Dropout(0.2)(query_embedded_sequences)
    query_cnn = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(d_query_embedded_sequences)
    d_query_cnn=Dropout(0.2)(query_cnn)
    queryrep=AttLayer()(Dense(200,activation='tanh')(d_query_cnn))
    query_encoder = Model([query_input], queryrep)
    queries_input = Input(shape=(MAXQ,MAXQL,), dtype='int32') 
    queryreps = TimeDistributed(query_encoder)(queries_input)
    userrep_query=AttLayer()(Dense(200,activation='tanh')(queryreps))
    
    title_input = Input(shape=(MAXTL,), dtype='int32')
    title_embedded_sequences = embedding_layer(title_input)
    d_title_embedded_sequences=Dropout(0.2)(title_embedded_sequences)
    title_cnn = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(d_title_embedded_sequences)
    d_title_cnn=Dropout(0.2)(title_cnn)
    titlerep=AttLayer()(Dense(200,activation='tanh')(d_title_cnn))
    title_encoder = Model([title_input], titlerep)
    titles_input = Input(shape=(MAXT,MAXTL,), dtype='int32') 
    titlereps = TimeDistributed(title_encoder)(titles_input)
    userrep_title=AttLayer()(Dense(200,activation='tanh')(titlereps))

    uservecs =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(vec) for vec in [userrep_news,userrep_title,userrep_query]],axis=1)
    l_atttv=AttLayer()(Dense(200,activation='tanh')(uservecs))
    
    candidates = keras.Input((1+npratio,MAX_SENT_LENGTH,), dtype='int32')
    candidate_vecs = TimeDistributed(news_encoder)(candidates)
    logits = keras.layers.dot([l_atttv, candidate_vecs], axes=-1)
    logits = keras.layers.Activation(keras.activations.softmax)(logits)
    model = Model([candidates,news_input,queries_input,titles_input], [logits])
    
    candidate_one = keras.Input((MAX_SENT_LENGTH,))
    candidate_one_vec = news_encoder([candidate_one,user_id])
    score = keras.layers.Activation(keras.activations.sigmoid)(
        keras.layers.dot([l_atttv, candidate_one_vec], axes=-1))
    model_test = keras.Model([candidate_one,news_input,queries_input,titles_input], score)
    model.compile(loss=['categorical_crossentropy'], optimizer=Adam(lr=0.001), metrics=['acc'])

    for epoch in range(2):
        traingen=generate_batch_data_random(train_candidate,train_label,train_userid, 30)
        model.fit_generator(traingen, epochs=1,steps_per_epoch=len(train_userid)//30)
        valgen=generate_batch_data(test_candidate,test_label,test_userid, 50)
        pred = model_test.predict_generator(valgen, steps=len(test_userid)//50,verbose=1)
        from sklearn.metrics import roc_auc_score
        all_auc=[]
        all_mrr=[]
        all_ndcg=[]
        all_ndcg2=[]
        for index in all_test_index:
            if np.sum(test_label[index[0]:index[1]])!=0 and index[1]<len(pred):
                all_auc.append(roc_auc_score(test_label[index[0]:index[1]],pred[index[0]:index[1],0]))
                all_mrr.append(mrr_score(test_label[index[0]:index[1]],pred[index[0]:index[1],0]))
                all_ndcg.append(ndcg_score(test_label[index[0]:index[1]],pred[index[0]:index[1],0],k=5))
                all_ndcg2.append(ndcg_score(test_label[index[0]:index[1]],pred[index[0]:index[1],0],k=10))
        results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
        print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))